<a href="https://colab.research.google.com/github/SaiidAmiri/AIS_Aufgabenstellung/blob/main/Said_Amiri_Aufgabenstellung_AIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0- Bibliotheken und Datensätze importieren**

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from google.colab import drive

In [2]:
# Mount Google Drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# Datensätze importieren
Kundendatei_path = '/gdrive/MyDrive/AIS_Management/Kundendatei.xlsx'
Beispielobjekte_path = '/gdrive/MyDrive/AIS_Management/Beispielobjekte.xlsx'
EP_Katalog_path = '/gdrive/MyDrive/AIS_Management/EP_Katalog.xlsx'
df_Kunden = pd.read_excel(Kundendatei_path)
df_Beispielobjekt = pd.read_excel(Beispielobjekte_path)
df_EP_Katalog = pd.read_excel(EP_Katalog_path)

**1. Vollständigkeitsprüfung**

In [4]:
def Vollständigkeit(df_Kunden, df_Beispielobjekt, model):
  """
    Prüft, ob typische Anlagen pro Gebäude aus den Beispielobjekte im Kundendatensatz enthalten sind.

    Vergleicht Kundenanlagen mit typischen Referenzanlagen anhand semantischer Ähnlichkeit
    (SentenceTransformer-Modell).

    Parameter:
        df_Kunden (pd.DataFrame): Kundendaten mit 'WirtEinh' und 'EQ-Klasse-Bezeichnung'.
        df_Beispielobjekt (pd.DataFrame): Referenzanlagen mit 'AKS-Bezeichnung'.
        model: Ein SentenceTransformer-Modell zur Vektor-Repräsentation der Anlagentexte.

    Rückgabe:
        pd.DataFrame: Tabelle mit Gebäude, Anlage, höchste berechnete Ähnlichkeit zur Kundenanlage und Status ('✅' oder '❌').
  """
  # Liste typischer Anlagen
  typische_anlagen = df_Beispielobjekt["AKS-Bezeichnung"].dropna().drop_duplicates()
  # Alle Gebäude
  gebaeude_ids = df_Kunden["WirtEinh"].drop_duplicates()
  # Lade und bereite Daten vor
  result = []

  # Pre-encode Referenzanlagen für Effizienz
  typische_anlagen_list = [str(a) for a in typische_anlagen]
  typische_embeddings = model.encode(typische_anlagen_list, convert_to_tensor=True)
  for gid in gebaeude_ids:
    # Kundenanlagen pro Gebäude
    gebaeude_anlagen = df_Kunden[df_Kunden["WirtEinh"] == gid]["EQ-Klasse-Bezeichnung"].dropna().unique()
    gebaeude_anlagen_list = [str(a) for a in gebaeude_anlagen]
    kunden_embeddings = model.encode(gebaeude_anlagen_list, convert_to_tensor=True)

    # Ähnlichkeitsmatrix: Kundenanlagen vs. Referenzanlagen
    cosine_similarities = util.cos_sim(kunden_embeddings, typische_embeddings)

    # max pro Spalte (jede Referenzanlage → höchster Match mit einer Kundenanlage)
    for idx, referenz in enumerate(typische_anlagen_list):
        max_similarity = cosine_similarities[:, idx].max().item()
        vorhanden = max_similarity > 0.7  # Schwellenwert (anpassbar)
        result.append({
            "Gebäude": gid,
            "Typische Anlage": referenz,
            "Maximale Ähnlichkeit": round(max_similarity, 2),
            "Status": "✅ Vorhanden" if vorhanden else "❌ Fehlend"
        })

  return pd.DataFrame(result)

In [7]:
# SentenceTransformer-Modell zur Vektor-Repräsentation der Anlagentexte
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Ergebnisdatei zur Vollständigkeitsprüfung
df_Vergleich = Vollständigkeit(df_Kunden, df_Beispielobjekt, model)
# Excel-Datei exportieren
file_path = '/gdrive/MyDrive/AIS_Management/Vollständigkeitsprüfung.xlsx'
df_Vergleich.to_excel(file_path, index=False)

**2. EP-Zuordnung**

In [6]:
def EP_Zuordnung(df_Kunden, df_EP_Katalog):
  """
    Ordnet jeder Kundenanlage die passendste Artikelnummer aus dem EP-Katalog zu.

    Nutzt TF-IDF-Vektorisierung und Cosine Similarity, um Kundenbeschreibungen mit
    Katalogtexten zu vergleichen.

    Parameter:
        df_Kunden (pd.DataFrame): Kundendaten mit Anlagenbezeichnungen.
        df_EP_Katalog (pd.DataFrame): EP-Katalog mit 'Artikelnummer' und 'Kurztext / Bezeichnung'.

    Rückgabe:
        pd.DataFrame: Erweiterter Kundendatensatz mit zugeordneter Artikelnummer und Kurztext.
  """
  # Nur Zeilen mit Artikelnummern
  df_EP_Katalog = df_EP_Katalog[df_EP_Katalog["Artikelnummer"].notna()]

  # Texte zusammenstellen
  kunden_texts = df_Kunden["EQ-Bezeichnung"].fillna('') + " " + \
               df_Kunden["EQ-Klasse-Bezeichnung"].fillna('') + " " + \
               df_Kunden["Anlagenausprägung"].fillna('')

  ep_texts = df_EP_Katalog["Kurztext / Bezeichnung"].fillna('')

  # TF-IDF-Vektoren berechnen
  vectorizer = TfidfVectorizer().fit(pd.concat([kunden_texts, ep_texts]))
  kunden_vecs = vectorizer.transform(kunden_texts)
  ep_vecs = vectorizer.transform(ep_texts)

  # Ähnlichkeiten berechnen
  similarity_matrix = cosine_similarity(kunden_vecs, ep_vecs)
  best_matches = similarity_matrix.argmax(axis=1)

  # Zuordnungen einfügen
  df_Kunden["Zugeordnete Artikelnummer"] = df_EP_Katalog.iloc[best_matches]["Artikelnummer"].values
  df_Kunden["Artikel Kurztext"] = df_EP_Katalog.iloc[best_matches]["Kurztext / Bezeichnung"].values

  return df_Kunden

In [9]:
# Ergebnisdatei zur EP-Zuordnung
df_EP_Zuordnung = EP_Zuordnung(df_Kunden, df_EP_Katalog)
# Excel-Datei exportieren
file_path = '/gdrive/MyDrive/AIS_Management/EP_Zuordnung.xlsx'
df_EP_Zuordnung.to_excel(file_path, index=False)